In [23]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_unixtime
import findspark

In [2]:
spark = SparkSession.builder.appName("Books").getOrCreate()

In [3]:
df = spark.read.csv("../data/books_rating.csv", header = True, inferSchema = True)

In [15]:
df.show(truncate = False)

+----------+-------------------------------------+-----+--------------+--------------------------------------+------------------+------------+-----------+------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [17]:
df.groupBy("Title").count().sort("count", ascending = False).show(truncate = False)

+----------------------------------------------------------------+-----+
|Title                                                           |count|
+----------------------------------------------------------------+-----+
|The Hobbit                                                      |22023|
|Pride and Prejudice                                             |20371|
|Atlas Shrugged                                                  |12513|
|Wuthering Heights                                               |10780|
|The Giver                                                       |7644 |
|Great Expectations                                              |7421 |
|Harry Potter and The Sorcerer's Stone                           |6796 |
|Of Mice and Men                                                 |6728 |
|Brave New World                                                 |6312 |
|Mere Christianity                                               |6053 |
|The Picture of Dorian Gray                        

In [24]:
df2 = df.select(from_unixtime("review/time")).show(30, truncate = False)

+-----------------------------------------------+
|from_unixtime(review/time, yyyy-MM-dd HH:mm:ss)|
+-----------------------------------------------+
|1999-10-23 02:00:00                            |
|2004-09-21 02:00:00                            |
|2004-03-09 01:00:00                            |
|2004-07-25 02:00:00                            |
|2005-02-10 01:00:00                            |
|2005-09-20 02:00:00                            |
|2004-11-11 01:00:00                            |
|2009-01-06 01:00:00                            |
|2008-05-04 02:00:00                            |
|2004-02-10 01:00:00                            |
|2001-06-02 02:00:00                            |
|2010-12-08 01:00:00                            |
|2009-07-23 02:00:00                            |
|2008-09-28 02:00:00                            |
|2005-05-26 02:00:00                            |
|2005-06-24 02:00:00                            |
|2005-06-20 02:00:00                            |


In [31]:
df.groupBy("Title").count().sort("count", ascending = False).show(8000, truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|Title                                                                                                                                                                                                                                                        |count|
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|The Hobbit                                                                                                                                                                                                           